In [8]:
# ======================================================================
# SECTION 1 – IMPORT & CONFIG
# ======================================================================

!pip install efficientnet_pytorch pytorch-grad-cam scikit-learn matplotlib tqdm

import torch, torch.nn as nn
from efficientnet_pytorch import EfficientNet
from torch.cuda.amp import autocast
import numpy as np, json, matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Device: {device}")


✅ Device: cuda


ERROR: Could not find a version that satisfies the requirement pytorch-grad-cam (from versions: none)
ERROR: No matching distribution found for pytorch-grad-cam


In [9]:
# ======================================================================
# SECTION 2 – MODEL DEFINITION
# ======================================================================

class ChestXrayClassifier(nn.Module):
    def __init__(self, num_classes=14, pretrained=False):
        super().__init__()
        self.backbone = EfficientNet.from_name('efficientnet-b0')
        num_features = self.backbone._fc.in_features
        # Same head as during training (Dropout + Linear)
        self.backbone._fc = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(num_features, num_classes)
        )
    def forward(self, x):
        return self.backbone(x)


In [10]:
# ======================================================================
# SECTION 3 – LOAD BOTH TRAINED MODELS
# ======================================================================

def load_checkpoint(path):
    print(f"Loading {path}")
    ckpt = torch.load(path, map_location=device, weights_only=False)
    model = ChestXrayClassifier().to(device)
    model.load_state_dict(ckpt["model_state_dict"])
    model.eval()
    return model, ckpt

model128_path = "experiments/chest_xray_20251022_093109/best_model.pth"
model224_path = "experiments/chest_xray_224x224_20251023_101202/best_model.pth"

model128, ckpt128 = load_checkpoint(model128_path)
model224, ckpt224 = load_checkpoint(model224_path)

print(f"✅ 128×128 Val AUC = {ckpt128['val_auc_macro']:.4f}")
print(f"✅ 224×224 Val AUC = {ckpt224['val_auc_macro']:.4f}")


Loading experiments/chest_xray_20251022_093109/best_model.pth
Loading experiments/chest_xray_224x224_20251023_101202/best_model.pth
✅ 128×128 Val AUC = 0.8024
✅ 224×224 Val AUC = 0.8265


In [11]:
# ======================================================================
# REBUILD TEST DATASET AND TEST LOADER (224×224 images)
# ======================================================================

import os, pandas as pd, numpy as np
from albumentations import Compose, Resize, Normalize
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
extract_path = "./chest_xray_data/"     # update if different

# ----------------------------------------------------
# 1️⃣  Define dataset class (same as during training)
# ----------------------------------------------------
class ChestXrayDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, num_classes=14):
        self.dataframe = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.num_classes = num_classes
        self.pathologies = [
            'Atelectasis', 'Consolidation', 'Infiltration', 
            'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis',
            'Effusion', 'Pneumonia', 'Pleural_Thickening', 
            'Cardiomegaly', 'Nodule', 'Mass', 'Hernia'
        ]

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx]['Image Index']
        # possible folders
        candidates = [os.path.join(self.img_dir, 'images', img_name)]
        for i in range(1, 13):
            candidates.append(os.path.join(self.img_dir, f'images_{i:03d}', 'images', img_name))
        path = next((p for p in candidates if os.path.exists(p)), None)
        if path is None: raise FileNotFoundError(img_name)
        img = np.array(Image.open(path).convert("RGB"))
        labels_str = self.dataframe.iloc[idx]['Finding Labels']
        y = np.zeros(len(self.pathologies), dtype=np.float32)
        if labels_str != "No Finding":
            for i, d in enumerate(self.pathologies):
                if d in labels_str: y[i] = 1.0
        if self.transform:
            img = self.transform(image=img)["image"]
        return img, torch.tensor(y)

# ----------------------------------------------------
# 2️⃣  Load DataFrame and build test split
# ----------------------------------------------------
data_entry = pd.read_csv(os.path.join(extract_path, "Data_Entry_2017.csv"))
test_list_path = os.path.join(extract_path, "test_list.txt")

if os.path.exists(test_list_path):
    test_images = pd.read_csv(test_list_path, header=None)[0].tolist()
    test_data = data_entry[data_entry["Image Index"].isin(test_images)].reset_index(drop=True)
else:
    from sklearn.model_selection import train_test_split
    _, test_data = train_test_split(data_entry, test_size=0.3, random_state=42)
print(f"✅ Loaded test split: {len(test_data):,} images.")

# ----------------------------------------------------
# 3️⃣  Define transforms (224×224 normalization)
# ----------------------------------------------------
val_transforms = Compose([
    Resize(224, 224),
    Normalize(mean=[0.485, 0.456, 0.406],
              std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# ----------------------------------------------------
# 4️⃣  Build DataLoader
# ----------------------------------------------------
BATCH_SIZE = 32
NUM_WORKERS = 0

test_dataset = ChestXrayDataset(test_data, img_dir=extract_path, transform=val_transforms)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                         shuffle=False, num_workers=NUM_WORKERS)

print(f"✅ test_loader ready ({len(test_loader)} batches of {BATCH_SIZE})")


✅ Loaded test split: 25,596 images.
✅ test_loader ready (800 batches of 32)


In [12]:
# ======================================================================
# SECTION 4 – COLLECT PREDICTIONS
# ======================================================================

def get_model_predictions(model, loader):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for imgs, lbls in tqdm(loader, desc="Predicting"):
            imgs = imgs.to(device)
            with autocast():
                out = model(imgs)
                probs = torch.sigmoid(out).cpu().numpy()
            preds.append(probs)
            labels.append(lbls.numpy())
    return np.vstack(preds), np.vstack(labels)

pred128, labels = get_model_predictions(model128, test_loader)
pred224, _ = get_model_predictions(model224, test_loader)

print(f"Shapes – 128:{pred128.shape}, 224:{pred224.shape}, Labels:{labels.shape}")


C:\Users\91850\AppData\Local\Temp\ipykernel_21316\798882183.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Predicting: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 800/800 [1:14:45<00:00,  5.61s/it]

Shapes – 128:(25596, 14), 224:(25596, 14), Labels:(25596, 14)


In [13]:
# ======================================================================
# SECTION 5 – ENSEMBLE EVALUATION
# ======================================================================

def evaluate_preds(y_true, y_pred):
    auc_macro = roc_auc_score(y_true, y_pred, average="macro")
    auc_weighted = roc_auc_score(y_true, y_pred, average="weighted")
    binary = (y_pred >= 0.5).astype(int)
    acc = accuracy_score(y_true.flatten(), binary.flatten())
    f1m = f1_score(y_true, binary, average="macro")
    f1w = f1_score(y_true, binary, average="weighted")
    print(f"AUC (macro): {auc_macro:.4f} | AUC (weighted): {auc_weighted:.4f}")
    print(f"Acc: {acc:.4f} | F1 macro: {f1m:.4f} | F1 weighted: {f1w:.4f}")
    return auc_macro, auc_weighted, acc, f1m, f1w

# Weighted fusion – favor 224×224 (0.6)
w1, w2 = 0.4, 0.6
ensemble_preds = np.clip(w1*pred128 + w2*pred224, 0, 1)
ensemble_metrics = evaluate_preds(labels, ensemble_preds)

# Save
res = {
    "Model_128x128": float(ckpt128["val_auc_macro"]),
    "Model_224x224": float(ckpt224["val_auc_macro"]),
    "Ensemble_AUC_macro": ensemble_metrics[0],
    "AUC_weighted": ensemble_metrics[1],
    "Accuracy": ensemble_metrics[2],
    "F1_macro": ensemble_metrics[3],
    "F1_weighted": ensemble_metrics[4],
}
with open("experiments/final_ensemble_results.json", "w") as f:
    json.dump(res, f, indent=4)
print("✅ Saved final ensemble metrics → experiments/final_ensemble_results.json")


AUC (macro): 0.7870 | AUC (weighted): 0.7659
Acc: 0.7374 | F1 macro: 0.2769 | F1 weighted: 0.3422
✅ Saved final ensemble metrics → experiments/final_ensemble_results.json


In [15]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import cv2, random, matplotlib.pyplot as plt
import numpy as np

# choose target convolution layer
target_layers = [model224.backbone._conv_head]

# NEW correct syntax (no use_cuda)
cam = GradCAM(model=model224, target_layers=target_layers)


In [16]:
def visualize_gradcam(image_path, predicted_class_idx):
    img = cv2.imread(image_path)
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    rgb_img = cv2.resize(rgb_img, (224, 224))
    input_tensor = torch.tensor(rgb_img/255.).permute(2,0,1).unsqueeze(0).float().to(device)

    targets = [ClassifierOutputTarget(predicted_class_idx)]
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0, :]
    visualization = show_cam_on_image(rgb_img/255., grayscale_cam, use_rgb=True)

    plt.imshow(visualization)
    plt.title(f"Grad‑CAM (class {predicted_class_idx})")
    plt.axis("off")
    plt.show()


In [20]:
# ======================================================================
# FIX - LOOK FOR NIH IMAGES IN ALL SUBDIRECTORIES
# ======================================================================
import os, glob, random, cv2
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import matplotlib.pyplot as plt
import numpy as np

# gather all .png or .jpg files from every subfolder like images_001/.../images
image_paths = []
for i in range(1, 13):
    folder = f"./chest_xray_data/images_{i:03d}/images"
    if os.path.exists(folder):
        image_paths.extend(glob.glob(os.path.join(folder, "*.png")))
        image_paths.extend(glob.glob(os.path.join(folder, "*.jpg")))

# sample 20 random images
samples = random.sample(image_paths, 20)
save_dir = "gradcam_visuals"
os.makedirs(save_dir, exist_ok=True)

print(f"Found {len(image_paths):,} total images, sampling 20 for Grad‑CAM...")

# run Grad‑CAM visualization save loop
for path in samples:
    fname = os.path.basename(path)
    img = cv2.imread(path)
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    rgb_img = cv2.resize(rgb_img, (224,224))
    input_tensor = torch.tensor(rgb_img/255.).permute(2,0,1).unsqueeze(0).float().to(device)
    targets = [ClassifierOutputTarget(10)]  # example class index: Cardiomegaly
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0, :]
    vis = show_cam_on_image(rgb_img/255., grayscale_cam, use_rgb=True)
    cv2.imwrite(f"{save_dir}/gradcam_{fname}", cv2.cvtColor(vis, cv2.COLOR_RGB2BGR))

print(f"✅ Saved 20 Grad‑CAM heatmaps → {save_dir}/")


Found 112,120 total images, sampling 20 for Grad‑CAM...
✅ Saved 20 Grad‑CAM heatmaps → gradcam_visuals/


In [21]:
# Example snippet to overlay label text on saved Grad‑CAM images
import cv2, os
for img in os.listdir("gradcam_visuals"):
    path = f"gradcam_visuals/{img}"
    vis = cv2.imread(path)
    label_text = "Cardiomegaly  (Pred: Positive)"
    cv2.putText(vis, label_text, (15, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
    cv2.imwrite(path, vis)
print("✅ Labeled gradcam visuals updated for presentation.")


✅ Labeled gradcam visuals updated for presentation.
